In [1]:
import sys
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import gc
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from keras.utils.vis_utils import plot_model
from sklearn.metrics.pairwise import cosine_similarity
import pickle
tqdm.pandas()

In [2]:
item_attributes = pd.read_csv("Dataset/item_attributes.csv")
item_attributes = item_attributes[item_attributes.columns[:905]]
item_attributes

,item_id,0,1,2,3,4,5,6,7,8,...,894,895,896,897,898,899,900,901,902,903
0,2,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,3,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,7,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,1,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23687,28140,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23688,28141,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23689,28142,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
class VariationalAutoEncoder(keras.Model):
    def __init__(self, inputShape, batchSize, latentSize):
        super(VariationalAutoEncoder, self).__init__()
        self.inputShape = inputShape
        self.batchSize = batchSize
        self.latentSize = latentSize
        self.input_layer = keras.Input(shape=self.inputShape)
        self.e2 = keras.layers.Dense(units=self.latentSize*4)(self.input_layer)
        self.b2 = tf.keras.layers.BatchNormalization()(self.e2)
        self.r2 = tf.keras.layers.LeakyReLU(alpha=0.3)(self.b2)
        self.d2 = tf.keras.layers.Dropout(0.2, seed = 42)(self.r2)
        self.z_mean = keras.layers.Dense(self.latentSize)(self.d2)
        self.z_log_sigma = keras.layers.Dense(self.latentSize)(self.d2)
        self.z = keras.layers.Lambda(self.sampling)([self.z_mean, self.z_log_sigma])
        self.encoder = keras.Model(self.input_layer, [self.z_mean, self.z_log_sigma, self.z], name='encoder')
        self.latent_inputs = keras.Input(shape=(self.latentSize,), name='z_sampling')
        self.e5 = keras.layers.Dense(units=self.latentSize*4)(self.latent_inputs)
        self.b5 = tf.keras.layers.BatchNormalization()(self.e5)
        self.r5 = tf.keras.layers.LeakyReLU(alpha=0.3)(self.b5)
        self.d5 = tf.keras.layers.Dropout(0.2, seed = 42)(self.r5)
        self.output_layer = keras.layers.Dense(self.inputShape[0], activation='sigmoid')(self.d5)
        self.decoder = keras.Model(self.latent_inputs, self.output_layer, name='decoder')
        self.output_layer = self.decoder(self.encoder(self.input_layer)[2])
        self.vae = keras.Model(self.input_layer, self.output_layer, name='vae_mlp')

    def sampling(self, args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], self.latentSize),
                              mean=0., stddev=0.1)
        return z_mean + K.exp(z_log_sigma) * epsilon

    def call(self, vector):
        z_mean, z_log_sigma, z = self.encoder(vector)
        reconstructed = self.decoder(z)
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_sigma - tf.square(z_mean) - tf.exp(z_log_sigma) + 1
        )
        self.add_loss(kl_loss)
        self.add_metric(kl_loss, name='kl_loss', aggregation='mean')
        return reconstructed

    def createEmbedding(self, vector):
        return self.encoder(vector)

In [4]:
dataset = item_attributes[item_attributes.columns[1:]].values

In [5]:
autoEncoder = VariationalAutoEncoder(inputShape=(dataset.shape[1],),
                                     batchSize=32,
                                     latentSize=64)
autoEncoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=keras.losses.MeanSquaredError())

In [6]:
dataset_tensor = tf.convert_to_tensor(dataset)
dataset_tensor.shape

TensorShape([23691, 904])

In [7]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1, cooldown=0)]

In [8]:
autoEncoder.fit(dataset_tensor, dataset_tensor,
                epochs=200,
                batch_size=32,
                validation_split=0.2,
                callbacks=[callbacks],
                verbose=1)

Epoch 1/200
593/593 [==============================] - 4s 5ms/step - loss: 0.1040 - kl_loss: 0.0814 - val_loss: 0.0178 - val_kl_loss: 0.0033 - lr: 0.0010

In [9]:
encoded_dataset = autoEncoder.createEmbedding(dataset_tensor)[2]
#encoded_dataset = np.hstack([encoded_dataset[0], encoded_dataset[1]])
encoded_dataset.shape

TensorShape([23691, 64])

In [10]:
similarity_matrix = cosine_similarity(encoded_dataset)

In [11]:
np.save('Dataset/ICM', encoded_dataset, allow_pickle=True)
np.save('Dataset/item_similarity_matrix', similarity_matrix, allow_pickle=True)